In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

In [3]:
# k
ks = [5, 10, 50]

score_function_dict = {
    "Recall"      : evaluators.recall,
    "Unpopularity": evaluators.unpopularity,
    "Unpopularity2": evaluators.unpopularity2,
    "Unpopularity3": evaluators.unpopularity3,
    "F1-score"    : evaluators.f1_score,
    "F1-score2"    : evaluators.f1_score2,
    "F1-score3"    : evaluators.f1_score3
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 30
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = False

search_range = 30

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma)
mymp = searches.Mymp(model, search_range)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [6]:
trainer.fit(n_batch, n_epoch)

epoch30 avg_loss:0.257: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:03<00:00, 74.25it/s]


In [ ]:
display(trainer.valid_scores)

In [7]:
trainer.valid(mymp, userwise)

  1%|█▏                                                                                                                                                                                            | 6/940 [00:00<00:32, 28.61it/s]

[tensor(17), tensor(41), tensor(123), tensor(130), tensor(97), tensor(55), tensor(178), tensor(14), tensor(230), tensor(103), tensor(153), tensor(238), tensor(171), tensor(265), tensor(100), tensor(165), tensor(74), tensor(167), tensor(136), tensor(67), tensor(224), tensor(92), tensor(158), tensor(54), tensor(185), tensor(141), tensor(373), tensor(277), tensor(277), tensor(277)]
[tensor(142), tensor(175), tensor(215), tensor(220), tensor(204), tensor(136), tensor(101), tensor(248), tensor(293), tensor(285), tensor(400), tensor(378), tensor(89), tensor(273), tensor(65), tensor(142), tensor(608), tensor(308), tensor(308), tensor(364), tensor(253), tensor(145), tensor(252), tensor(466), tensor(289), tensor(191), tensor(197), tensor(207), tensor(114), tensor(38)]
[tensor(23), tensor(183), tensor(390), tensor(1), tensor(28), tensor(603), tensor(47), tensor(3), tensor(543), tensor(543), tensor(543), tensor(23), tensor(23), tensor(505), tensor(593), tensor(52), tensor(533), tensor(533), tenso

  1%|█▊                                                                                                                                                                                            | 9/940 [00:00<00:33, 27.65it/s]

[tensor(174), tensor(219), tensor(180), tensor(185), tensor(171), tensor(349), tensor(359), tensor(281), tensor(165), tensor(165), tensor(269), tensor(469), tensor(358), tensor(439), tensor(167), tensor(222), tensor(65), tensor(187), tensor(287), tensor(325), tensor(154), tensor(199), tensor(199), tensor(577), tensor(52), tensor(321), tensor(558), tensor(317), tensor(209), tensor(539)]
[tensor(96), tensor(76), tensor(207), tensor(243), tensor(87), tensor(366), tensor(174), tensor(195), tensor(531), tensor(35), tensor(454), tensor(52), tensor(236), tensor(100), tensor(99), tensor(524), tensor(409), tensor(29), tensor(146), tensor(342), tensor(179), tensor(919), tensor(242), tensor(28), tensor(710), tensor(30), tensor(312), tensor(312), tensor(141), tensor(21)]
[tensor(18), tensor(18), tensor(77), tensor(43), tensor(226), tensor(92), tensor(84), tensor(210), tensor(12), tensor(227), tensor(159), tensor(77), tensor(447), tensor(484), tensor(338), tensor(338), tensor(147), tensor(70), tens

  2%|███▍                                                                                                                                                                                         | 17/940 [00:00<00:29, 31.02it/s]

...

 63%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                    | 596/940 [00:20<00:12, 27.04it/s]

[tensor(13), tensor(64), tensor(225), tensor(314), tensor(122), tensor(238), tensor(442), tensor(442), tensor(73), tensor(73), tensor(468), tensor(402), tensor(203), tensor(37), tensor(17), tensor(633), tensor(29), tensor(56), tensor(10), tensor(55), tensor(50), tensor(708), tensor(571), tensor(407), tensor(763), tensor(112), tensor(120), tensor(631), tensor(350), tensor(88)]
[tensor(23), tensor(17), tensor(82), tensor(4), tensor(87), tensor(47), tensor(39), tensor(154), tensor(152), tensor(42), tensor(42), tensor(87), tensor(174), tensor(222), tensor(211), tensor(246), tensor(285), tensor(151), tensor(137), tensor(411), tensor(77), tensor(299), tensor(115), tensor(115), tensor(404), tensor(463), tensor(165), tensor(80), tensor(228), tensor(352)]
[tensor(8), tensor(6), tensor(452), tensor(12), tensor(688), tensor(224), tensor(600), tensor(29), tensor(235), tensor(166), tensor(282), tensor(564), tensor(70), tensor(547), tensor(755), tensor(506), tensor(128), tensor(72), tensor(541), ten

 64%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                    | 598/940 [00:21<00:12, 28.45it/s]


[tensor(283), tensor(548), tensor(526), tensor(267), tensor(383), tensor(12), tensor(553), tensor(777), tensor(29), tensor(35), tensor(52), tensor(211), tensor(634), tensor(376), tensor(45), tensor(275), tensor(273), tensor(556), tensor(369), tensor(268), tensor(91), tensor(63), tensor(296), tensor(314), tensor(789), tensor(272), tensor(337), tensor(63), tensor(79), tensor(283)]


KeyboardInterrupt: 

In [ ]:
trainer.valid_scores

In [ ]:
mp2 = searches.MutualProximity(model, gaussian)

In [ ]:
trainer.valid(mp, recall)

In [ ]:
trainer.valid_scores